In [2]:
import requests
import camelot
import datetime
import os
import pandas as pd
from deta import Deta
from dotenv import load_dotenv
import constants

load_dotenv()

deta = Deta(os.environ.get("DETA_PROJECT_KEY"))

data = deta.Drive("data")


In [3]:
url = 'https://www.dropbox.com/s/ckijmipu33z3feg/HourlyReport.pdf?dl=1'
r = requests.get(url, allow_redirects=True)
open('hourlyreport.pdf', 'wb').write(r.content)

1042262

In [4]:
# tables = camelot.read_pdf('hourlyreport.pdf', flavor='stream', pages='1', columns=['41, 57, 75, 97.5, 115.5, 138, 160, 187, 202, 220, 238, 256, 279, 301, 323, 342, 360, 381, 400, 425, 443, 465, 483, 501, 519, 537, 559.5, 577.5, 600, 627.5, 652, 667, 690, 706.5, 730, 753, 778, 798, 822, 839.5, 855.5, 881, 906, 930, 956, 978'])

In [12]:
tables = camelot.read_pdf('hourlyreport.pdf', pages= '1', flavor='stream', table_areas=['8,410,1000,50'])
tables[0].df

,0,1,2,3,4,5,6,7,8,9,...,37,38,39,40,41,42,43,44,45,46
0,11/17/23,13,6,77,14,51,20,128,3,21,...,19,12,3,0,16,0,2,6,10,3
1,,12,9,71,9,37,18,108,2,18,...,15,11,1,0,18,0,3,3,7,3
2,,11,11,62,7,28,18,90,4,16,...,10,7,3,0,22,1,1,1,6,3
3,,10,7,51,8,21,15,72,1,12,...,6,3,0,0,14,2,0,2,6,3
4,,9,14,44,3,13,17,57,4,11,...,11,10,1,0,13,2,0,1,6,3
5,,8,9,30,2,10,11,40,0,7,...,2,2,1,0,16,2,1,1,5,3
6,,7,6,21,0,8,6,29,0,7,...,10,7,0,0,13,2,0,1,5,3
7,,6,2,15,0,8,2,23,2,7,...,7,4,0,0,9,2,0,2,4,3
8,,5,1,13,1,8,2,21,0,5,...,10,9,0,0,9,2,0,1,4,3
9,,4,3,12,0,7,3,19,1,5,...,13,12,0,0,8,2,0,3,4,3


In [6]:
df = tables[0].df.reset_index(drop=True)
df.columns = constants.column_names
df.dateflg = df.dateflg[df.dateflg.str.strip() != '']
df.dateflg = df.dateflg.ffill()
for column in df.columns.tolist():
    if column in ['dateflg']:
        continue
    df[column] = df[column].astype('float').astype('int')
df["ds"] = pd.to_datetime(
    df["dateflg"] + " " + (df["timeflg"] - 1).astype(str) + ":00", format='mixed') + datetime.timedelta(hours=1)
df = df.set_index('ds').reset_index().drop(['dateflg','timeflg'], axis=1)


In [7]:
df.head()

,ds,INFLOW_STRETCHER,Infl_Stretcher_cum,INFLOW_AMBULATORY,Infl_Ambulatory_cum,Inflow_Total,Inflow_Cum_Total,INFLOW_AMBULANCES,Infl_Ambulances_cum,FLS,...,AMBVERT1,AMBVERTTBS,QTrack_TBS,Garage_TBS,RAZ_CONS_MORE2H,RAZ_IMCONS_MORE4H,RAZ_XRAY_MORE2H,RAZ_CT_MORE2H1,PSYCH1,PSYCH_WAITINGADM
0,2023-11-17 13:00:00,6,77,14,51,20,128,3,21,0,...,19,12,3,0,16,0,2,6,10,3
1,2023-11-17 12:00:00,9,71,9,37,18,108,2,18,0,...,15,11,1,0,18,0,3,3,7,3
2,2023-11-17 11:00:00,11,62,7,28,18,90,4,16,0,...,10,7,3,0,22,1,1,1,6,3
3,2023-11-17 10:00:00,7,51,8,21,15,72,1,12,0,...,6,3,0,0,14,2,0,2,6,3
4,2023-11-17 09:00:00,14,44,3,13,17,57,4,11,1,...,11,10,1,0,13,2,0,1,6,3


In [8]:
df.to_csv('current.csv', index=False)

In [9]:
data.put(name='current.csv', path='current.csv')

'current.csv'